In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()

In [14]:
# build hte vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)} 
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [88]:
# build the dataset

block_size = 3 # context-length
X, Y = [], []
for w in words:
    # print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and expand

X = torch.tensor(X)
Y = torch.tensor(Y)


In [89]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [29]:
C = torch.randn((27, 2))

In [30]:
C[1]

tensor([0.7295, 0.5210])

In [64]:
emb = C[X]
emb.shape

torch.Size([16, 3, 2])

In [65]:
# hidden layer
# input size for each neuron is 6 and number of neurons is random
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [68]:
# final layer
W2 = torch.randn(100, 27)
b2 = torch.randn(27)
logits = h @ W2 + b2

In [70]:
logits.shape

torch.Size([16, 27])

In [72]:
# softmax
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
prob.shape

torch.Size([16, 27])

In [75]:
prob[torch.arange(16), Y]

tensor([9.5539e-05, 1.4037e-14, 4.7028e-15, 1.1544e-15, 2.5996e-09, 1.4222e-08,
        1.6206e-09, 1.4362e-11, 4.0513e-08, 2.9901e-10, 1.7141e-08, 2.8413e-12,
        1.5377e-08, 8.7145e-05, 1.2169e-09, 1.3060e-10])

In [123]:
#---------- redefinition ----------
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [91]:
sum(p.nelement() for p in parameters)

3481

In [124]:
for p in parameters:
    p.requires_grad = True

In [137]:
for _ in range(10000):
    # mini batch

    ix = torch.randint(0, X.shape[0], (32, ))
    # forward pass
    emb = C[X[ix]] #[16, 3, 2]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # [16, 100]
    logits = h @ W2 + b2 # [16, 27]
    loss = F.cross_entropy(logits, Y[ix])

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    for p in parameters:
        p.data += -0.01 * p.grad
print(loss.item())

2.403048276901245


In [138]:
emb = C[X] #[16, 3, 2]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # [16, 100]
logits = h @ W2 + b2 # [16, 27]
loss = F.cross_entropy(logits, Y)
loss

tensor(2.2908, grad_fn=<NllLossBackward0>)

In [87]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])